In [1]:
from pathlib import Path

import pytz

from post_processing.utils.fpod_utils import csv_folder, cpod2aplose, fpod2aplose, actual_data, meta_cut_aplose
from post_processing.utils.core_utils import json2df

### Load data
Import your raw FPOD or CPOD data. All files for one site must be stored in the same folder.
You also need to import your metadata file.

In [2]:
pod_files = Path(r"U:\Cetiroise")
path = csv_folder(pod_files) #Path to your data folder.
json = Path(r"C:\Users\fouinel\Downloads\deployment_cetiroise.json") #Path to your metadata file.

print(path.head())
df_0 = path.dropna()

metadatax = json2df(json_path=json)
metadatax["deploy.name"] = (metadatax["site.name"].astype(str) + "_" +  metadatax["campaign.name"].astype(str))

                                                File  podN          ChunkEnd  \
0  CETIROISEPHASE4POINTC 2023 02 03 FPOD_6669 fil...  6669  07/02/2023 05:34   
1  CETIROISEPHASE4POINTC 2023 02 03 FPOD_6669 fil...  6669  07/02/2023 05:35   
2  CETIROISEPHASE4POINTC 2023 02 03 FPOD_6669 fil...  6669  07/02/2023 05:36   
3  CETIROISEPHASE4POINTC 2023 02 03 FPOD_6669 fil...  6669  07/02/2023 22:40   
4  CETIROISEPHASE4POINTC 2023 02 03 FPOD_6669 fil...  6669  24/02/2023 15:32   

     Minute  DPM  MinsOn      deploy.name  
0  64748494    1       1  Point C_Phase 4  
1  64748495    1       1  Point C_Phase 4  
2  64748496    1       1  Point C_Phase 4  
3  64749520    1       1  Point C_Phase 4  
4  64773572    1       1  Point C_Phase 4  


In [3]:
d_beg_end = actual_data(df_0, col_timestamp="ChunkEnd")
d_beg_end.to_csv(r"U:\Deb_Fin_CETIROISE.csv", index=False)

In [4]:
df_1 = df_0[df_0["DPM"] !=0 ]

### APLOSE format
#### *C-POD*
Use cpod2aplose if you are managing C-POD data.

In [ ]:
df_aplose = cpod2aplose(df_1, pytz.utc, "Site D Simone", "Commerson")
print(df_aplose.head())

#### *F-POD*
Use fpod2aplose if you are managing F-POD data.

In [5]:
df_aplose = fpod2aplose(df_1, pytz.utc, "CETIROISE", "Marsouin")
print(df_aplose.head())

     dataset filename  start_time  end_time  start_frequency  end_frequency  \
0  CETIROISE                    0        60                0              0   
1  CETIROISE                    0        60                0              0   
2  CETIROISE                    0        60                0              0   
3  CETIROISE                    0        60                0              0   
4  CETIROISE                    0        60                0              0   

  annotation annotator                start_datetime  \
0   Marsouin      FPOD  2023-02-07T05:34:00.000+0000   
1   Marsouin      FPOD  2023-02-07T05:35:00.000+0000   
2   Marsouin      FPOD  2023-02-07T05:36:00.000+0000   
3   Marsouin      FPOD  2023-02-07T22:40:00.000+0000   
4   Marsouin      FPOD  2023-02-24T15:32:00.000+0000   

                   end_datetime  is_box      deploy.name  
0  2023-02-07T05:35:00.000+0000       0  Point C_Phase 4  
1  2023-02-07T05:36:00.000+0000       0  Point C_Phase 4  
2  2023-02-

### Clean your dataset
Remove useless lines, recorder outside the instrument submersion. Export your file to the aplose format. You can change the name of the file to match the project you are working on.

In [6]:
cleared = meta_cut_aplose(df_aplose, metadatax)
cleared.to_csv(r"U:\APLOSE_CETIROISE_pos.csv", index=False)